<a href="https://colab.research.google.com/github/zephyris/tryptag/blob/main/examples/open_cell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Install tryptag module

#@markdown Install the `tryptag` module using `pip`.

!pip install git+https://github.com/zephyris/tryptag

In [ ]:
#@title Setup tryptag

#@markdown The `tryptag` module handles all data retrieval.

# import and set up TryTag instance
from tryptag import TrypTag
tryptag = TrypTag()

# import skimage and numpy, for handling and displaying the images
import skimage
import numpy

# filter matplotlib warnings, used by skimage for image display
import warnings
warnings.filterwarnings( "ignore", module = "matplotlib\..*" )

# function for quickly displaying some images
def quickshow_images(images):
  # show all images
  print("All images")
  skimage.io.imshow_collection(images)
  skimage.io.show()
  print("")

  # show individual channels
  image_types = ["Phase contrast", "mNG fluorescence", "DNA fluorescence", "Thredholded phase contrast", "Thresholded DNA"]
  for image_index, image in enumerate(images):
    print(image_types[image_index])
    skimage.io.imshow(image)
    skimage.io.show()
    print("")

In [ ]:
#@title Open images for a cell

#@markdown Cells are opened using their `gene_id` (from [TriTrypDB](https://tritrypdb.org)) and tagged `terminus`, which must be `"n"` or `"c"`.
gene_id = "Tb927.9.8570" #@param {type:"string"}
terminus = "n" #@param ["n", "c"]
#@markdown Most cell lines have multiple fields of view and many cells per field of view. You can choose these using `field_index` and `cell_index`.
field_index = 2 #@param {type:"integer"}
cell_index = 8 #@param {type:"integer"}

#@markdown The `tryptag.open_cell` returns a list of 5 `skimage` images, commonly referred to as `[pha, mng, dna, pth, dth]`:
#@markdown 0. `pth` Phase contrast
#@markdown 1. `mng` mNG (tagged protein) fluorescence
#@markdown 2. `dna` Hoechst 33342 (DNA stain) fluorescence
#@markdown 3. `pth` Thresholded phase contrast
#@markdown 4. `dth` Thresholded DNA stain

#@markdown The first time you open a cell it will take some time for `tryptag` to download the necessary data from [Zenodo](https://zenodo.org).
#@markdown The data is saved in a cache directory, so the next time you open any cell for this cell line it will be far quicker.
#@markdown Data is in 96 well plate groupings and download time will depend on how many images were captured for that plate - all this data will be cached.
#@markdown In Google Colab this cache will (unfortunately) be deleted at the end of each session, but on your computer would be stored permanently.

# open the cell
cell_images = [pha, mng, dna, pth, dth] = tryptag.open_cell(gene_id, terminus, field_index, cell_index)

# show the images
quickshow_images(cell_images)

In [ ]:
#@title Image processing

#@markdown The opened `skimage` images can be processed, analysed and/or saved using `skimage`.
#@markdown For example using `skimage.restoration.rolling_ball` to generate a background estimate which can be subtracted.

#@markdown `skimage` images are `numpy` 2D `ndarray` objects, and can also be handled using `numpy` tools.
#@markdown This includes simple mathematical operations like addition, subtraction, multiplication, etc.
#@markdown For example, using `numpy` to measure median background, then subtract this from the image.

# mng
print("mNG fluorescence")
skimage.io.imshow(mng)
skimage.io.show()
print("")

# mng, minus median
print("... minus the image median and clipped to positive values")
mng_bgs = mng - numpy.median(mng)
mng_bgs[mng_bgs < 0] = 0
skimage.io.imshow(mng_bgs)
skimage.io.show()
print("")

# mng, with rolling ball filter
print("... with rolling ball filter, radius 5 px")
mng_rb = skimage.restoration.rolling_ball(mng, radius = 5)
skimage.io.imshow(mng - mng_rb)
skimage.io.show()
print("")

# mng, with rolling ball filter
print("... masked with phase contrast thresholded image")
mng_ma = mng * pth / 255
skimage.io.imshow(mng_ma)
skimage.io.show()

In [ ]:
#@title Open field of view

#@markdown Fields of view can also be opened from their `gene_id`, tagged `terminus`, and `field_index`.
gene_id = "Tb927.9.8570" #@param {type:"string"}
terminus = "n" #@param ["n", "c"]
field_index = 1 #@param {type:"integer"}

#@markdown Like `tryptag.open_cell`, `tryptag.open_field` returns a list of 5 `skimage` images.

# open the field
field_images = tryptag.open_field(gene_id, terminus, field_index)

# show the images
quickshow_images(field_images)

In [ ]:
#@title Advanced usage

#@markdown Custom field of view images can be injected into the opening of cell images using `open_cell_custom`.
#@markdown This is particularly useful if doing computationally costly image modifications prior to analysing all of the cells in a field of view.

gene_id = "Tb927.9.8570" #@param {type:"string"}
terminus = "n" #@param ["n", "c"]
field_index = 1 #@param {type:"integer"}
cell_index = 8 #@param {type:"integer"}

#@markdown Select which changes to make:
invert_pha = True #@param {type:"boolean"}
rolling_ball_mng = True #@param {type:"boolean"}
median_subtract_dna = True #@param {type:"boolean"}
size_filter_dth = True #@param {type:"boolean"}
#@markdown Any alterations to the thresholded phase contrast image might change where to find cells.

# open field of view
field_images = tryptag.open_field(gene_id, terminus, field_index)

# setup custom images list
custom_images = [None] * 5

# do modifications:
# invert phase contrast image
if invert_pha:
  custom_images[0] = 2**16 - field_images[0]

# rolling ball subtraction for mng
if rolling_ball_mng:
  custom_images[1] = field_images[1] - skimage.restoration.rolling_ball(field_images[1], radius = 5)

# median subtraction for dna
if median_subtract_dna:
  custom_images[2] = field_images[2] - numpy.median(field_images[2])

# size filter dth
if size_filter_dth:
  field_images[4] = skimage.measure.label(field_images[4])
  custom_images[4] = skimage.morphology.remove_small_objects(field_images[4], min_size = 250)

# open cell images, using modified field images
cell_images = tryptag.open_cell_custom(gene_id, terminus, field_index, cell_index, images = custom_images)

#@markdown More advanced analysis can alter or generate a new thresholded phase contrast image.
#@markdown This, along with custom `fill_centre`, `crop_centre` and `angle` parameters for cell locations, can be passed to `open_cell_custom`.

quickshow_images(cell_images)

In [ ]:
#@title Handling errors

#@markdown If you try to load a `gene_id`, `terminus`, `field_index` or `cell_index` which does not exist then you'll get an error.

bad_gene_id = "Tb927.12.8570" #@param {type:"string"}
bad_terminus = "o" #@param ["n", "c"]
bad_field_index = 15 #@param {type:"integer"}
bad_cell_index = 120 #@param {type:"integer"}

#@markdown Make sure you:
#@markdown 1. either use the `try` `except` syntax,

print("Try except method:")
try:
  images = tryptag.open_cell(bad_gene_id, bad_terminus, bad_field_index, bad_cell_index)
except:
  print("Oops, one of your query values does not exist!")
print("")

#@markdown 2. or check that the necessary values in the `tryptag.gene_list` object exist.

#@markdown If checking against `tryptag.gene_list` then make sure that data for a target `gene_id` and `terminus` has been retrieved using `fetch_data`.
#@markdown `tryptag` only knows how many fields of view and cells there are after fetching the microscopy data.

print("Manual gene_list check method:")
if bad_gene_id in tryptag.gene_list:
  if bad_terminus in tryptag.gene_list[gene_id]:
    # fetch the data for this gene_id and terminus
    tryptag.fetch_data(bad_gene_id, bad_terminus)
    if bad_field_index < len(tryptag.gene_list[bad_gene_id][bad_terminus]["cells"]):
      if bad_cell_index < len(tryptag.gene_list[bad_gene_id][bad_terminus]["cells"][bad_field_index]):
        images = tryptag.open_cell(bad_gene_id, bad_terminus, bad_field_index, bad_cell_index)
      else:
        print("cell_index", bad_cell_index, "out of range")
    else:
      print("field_index", bad_field_index, "out of range")
  else:
    print("data for terminus", bad_terminus, "not found")
else:
  print("data for gene_id", bad_gene_id, "not found")